In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [4]:
# df =pd.read_csv('../../data/more_features/more_feats_correlated_testing_noanem_noisy_6.csv')
# df = df.fillna(-1)
# df.head()
train_df = pd.read_csv('../../data/more_features/train_sets/train_set_noisy_4.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,10.583945,-1.000000,-1.000000,-1.000000,-1.000000,77.825790,86.687181,4.079860,1,1.839253,62.063817,51.442666,10.135274,7.181275,50.494408,31.751835,-1.000000,2.0
1,6.406548,60.114661,-1.000000,0.708135,293.762868,81.158569,-1.000000,2.368160,0,-1.000000,-1.000000,-1.000000,42.075158,-1.000000,-1.000000,19.219645,-1.000000,7.0
2,6.730334,193.694103,-1.000000,-1.000000,385.876048,77.647711,135.466128,2.600334,0,1.468012,50.555900,72.032199,48.297182,11.485545,47.471635,20.191001,35.106125,3.0
3,8.176067,81.078814,-1.000000,3.595544,-1.000000,77.129395,150.507825,3.180137,0,-1.000000,8.466432,107.796946,10.421249,27.086345,-1.000000,24.528201,-1.000000,7.0
4,10.848484,173.267380,1.477827,1.940069,108.930346,81.115198,-1.000000,4.012251,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,32.545451,-1.000000,6.0


In [5]:
test_df = pd.read_csv('../../data/more_features/train_sets/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.391136,-1.000000,1.504298,5.058881,467.197112,96.252926,217.860499,3.550376,1,-1.000000,36.703810,122.310168,49.897027,20.353251,-1.000000,34.173407,46.631388,6
1,10.355048,272.506966,-1.000000,0.000000,128.706486,101.912313,-1.000000,3.048223,1,-1.000000,-1.000000,-1.000000,29.327349,-1.000000,-1.000000,31.065144,-1.000000,2
2,11.159385,-1.000000,-1.000000,0.000000,-1.000000,103.395616,77.212369,3.237870,0,-1.000000,8.690122,65.554731,19.167967,7.239049,-1.000000,33.478155,-1.000000,2
3,8.179735,140.876632,5.484515,-1.000000,303.740826,99.877458,155.573175,2.456931,0,-1.000000,41.123526,119.433840,39.384848,6.394235,-1.000000,24.539204,51.219053,5
4,9.916825,-1.000000,3.909810,-1.000000,-1.000000,90.543986,189.347916,3.285748,0,1.501433,34.905667,59.177001,58.538524,22.843594,139.245204,29.750475,-1.000000,5


In [6]:
train_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [7]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [8]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [9]:
for steps in [int(19e6), int(18.5e6), int(19.5e6)]:
#for steps in [int(7.5e6), int(7.8e6), int(8.3e6)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/with_correlated_fts/constant_test_set/dqn_noisy_4_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.7      |
|    ep_rew_mean      | -0.7     |
|    exploration_rate | 0.86     |
|    success_rate     | 0.15     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 671      |
|    time_elapsed     | 416      |
|    total_timesteps  | 279627   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 9.84     |
|    n_updates        | 57406    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.95     |
|    ep_rew_mean      | -0.82    |
|    exploration_rate | 0.707    |
|    success_rate     | 0.09     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 568      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.95     |
|    ep_rew_mean      | 0.08     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.61     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 499      |
|    time_elapsed     | 12941    |
|    total_timesteps  | 6459401  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0741   |
|    n_updates        | 1602350  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.85     |
|    ep_rew_mean      | 0.08     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.62     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 503      |
|    time_elapsed     | 13627    |
|    total_timesteps  | 6859509  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.93     |
|    ep_rew_mean      | 0.12     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.6      |
| time/               |          |
|    episodes         | 3100000  |
|    fps              | 526      |
|    time_elapsed     | 23753    |
|    total_timesteps  | 12513515 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.181    |
|    n_updates        | 3115878  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.96     |
|    ep_rew_mean      | 0.26     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.69     |
| time/               |          |
|    episodes         | 3200000  |
|    fps              | 527      |
|    time_elapsed     | 24479    |
|    total_timesteps  | 12921329 |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.46     |
|    ep_rew_mean      | 0.1      |
|    exploration_rate | 0.05     |
|    success_rate     | 0.66     |
| time/               |          |
|    episodes         | 4600000  |
|    fps              | 513      |
|    time_elapsed     | 36262    |
|    total_timesteps  | 18638706 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0254   |
|    n_updates        | 4647176  |
----------------------------------
using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.88     |
|    ep_rew_mean      | -0.84    |
|    exploration_rate | 0.856    |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 893      |
|    t

KeyboardInterrupt: 

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_missing_1_7500000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [ ]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [ ]:
test_df.y_pred.value_counts()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)